In [10]:
#importar librerias
import numpy as np
import pandas as pd
import pandapower as pp
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

pd.set_option('display.float_format', lambda x: '{:.4f}'.format(x))


#crear la red a analizar
#red vacia para sistema 2
red2 = pp.create_empty_network()

#crarción de las barras del sistema 2
barra_1 = pp.create_bus(red2, vn_kv=110, name="Barra 1")
barra_2 = pp.create_bus(red2, vn_kv=220, name="Barra 2")
barra_1A = pp.create_bus(red2, vn_kv=220, name="Barra 1A")
barra_1B = pp.create_bus(red2, vn_kv=220, name="Barra 1B")
barra_2A = pp.create_bus(red2, vn_kv=220, name="Barra 2A")
barra_2B = pp.create_bus(red2, vn_kv=220, name="Barra 2B")
barra_3A = pp.create_bus(red2, vn_kv=220, name="Barra 3A")

#añadir tranformador entre barra 1 y 2
pp.create_transformer(red2, barra_2, barra_1, std_type="100 MVA 220/110 kV")
#añadir el generador en la barra 1 y que dicha barra sea la referencia
pp.create_ext_grid(red2, barra_2, va_pu=1.0, name="Slack")

#crear las lineas
pp.create_line(red2, barra_2, barra_1A, length_km=10.0, std_type="NAYY 4x150 SE", name="L2-1A")
pp.create_line(red2, barra_2, barra_1B, length_km=10.0, std_type="NAYY 4x150 SE", name="L2-1B")
pp.create_line(red2, barra_1A, barra_2A, length_km=15.0, std_type="NAYY 4x150 SE", name="L1A-2A")
pp.create_line(red2, barra_2A, barra_3A, length_km=20.0, std_type="NAYY 4x150 SE", name="L2A-3A")
pp.create_line(red2, barra_1B, barra_2B, length_km=30.0, std_type="NAYY 4x150 SE", name="L1B-2B")
pp.create_line(red2, barra_2B, barra_3A, length_km=15.0, std_type="NAYY 4x150 SE", name="L2B-3A")

#crear las cargas
pp.create_load(red2, barra_1A, p_mw=30.0, q_mvar=20.0, name="Carga Barra 1A")
pp.create_load(red2, barra_2A, p_mw=52.5, q_mvar=35.0, name="Carga Barra 2A")
pp.create_load(red2, barra_3A, p_mw=22.5, q_mvar=15.0, name="Carga Barra 3A")
pp.create_load(red2, barra_1B, p_mw=15.0, q_mvar=10.0, name="Carga Barra 1B")
pp.create_load(red2, barra_2B, p_mw=90.0, q_mvar=60.0, name="Carga Barra 2B")


pp.runpp(red2, algorithm='nr', numba=False,max_iteration=1000)



red2.res_line.index = red2.line.name
red2.res_bus.index = red2.bus.name

print(red2.res_bus)
print(red2.res_line)



          vm_pu  va_degree      p_mw   q_mvar
name                                         
Barra 1  1.0000    -0.0019    0.0000   0.0000
Barra 2  1.0000     0.0000 -214.1341 250.4655
Barra 1A 0.9965    -0.3539   30.0000  20.0000
Barra 1B 0.9978    -0.3699   15.0000  10.0000
Barra 2A 0.9926    -0.7364   52.5000  35.0000
Barra 2B 0.9897    -0.9318   90.0000  60.0000
Barra 3A 0.9909    -0.9808   22.5000  15.0000
        p_from_mw  q_from_mvar   p_to_mw  q_to_mvar  pl_mw   ql_mvar  \
name                                                                   
L2-1A    118.3347    -117.5735 -117.3225    78.4137 1.0123  -39.1598   
L2-1B     95.7443    -132.9160  -94.8009    93.6812 0.9434  -39.2348   
L1A-2A    87.3225     -98.4137  -86.5198    39.8355 0.8027  -58.5782   
L2A-3A    34.0198     -74.8355  -33.8075    -3.1523 0.2123  -77.9878   
L1B-2B    79.8009    -103.6812  -78.7208   -13.4786 1.0802 -117.1598   
L2B-3A   -11.2792     -46.5214   11.3075   -11.8477 0.0282  -58.3691   

        i